# Monitoring dynamism of Python and R meetup groups in Ireland II

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import dateutil
import json
import urllib2
import time
import math
import plotly
import plotly.plotly as py 
import plotly.tools as tls
from plotly.graph_objs import * #Figure, Data, Layout 
import numpy as np


# Data Collection and Processing using the Meetup API 

In [2]:

eventsL = []
for name in ["DublinR","Cork-Ireland-R-Users-Group","pythonireland","PyLadiesDublin"]: 
        URL = "https://api.meetup.com/2/events.json/?group_urlname=" + name + "&key=95c3a7577a477a7d105a70197b5755"
        group = json.load(urllib2.urlopen(URL))
        eventsL.append(group)
        
#function used to return a dataframe giving information about each events: fgroup_name, date, number of yes_rsvp
def get_event_details(group_urlname,URL):
    group = json.load(urllib2.urlopen(URL))
    eventr= DataFrame()
    LEN = len(group["results"])
    event_name = []
    group_name = []
    event_date_ms = []
    event_yes_rsvp =[]
    status =[]

    rsvp_limit = []
    for i in range(0,LEN):
        event_name.append(group["results"][i]["name"])
        group_name.append(group["results"][i]["group"]["urlname"])
        event_date_ms.append(group["results"][i]["time"])
        event_yes_rsvp.append(group["results"][i]["yes_rsvp_count"])
        #waiting_list.append(group["results"][i]["waitlist_count"])
        try:
            rsvp_limit.append(group["results"][i]["rsvp_limit"])
        except KeyError: 
            rsvp_limit.append(0)
        status.append(group["results"][i]["status"])
    
    event_date = [x / 1000 for x in event_date_ms]
    events= DataFrame()
    events["Event Name"] = Series(event_name)
    events["Group Name"] = Series(group_name)
    events["Date"] = map(lambda date: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(date)),event_date)
    events["Number of yes_rsvp"] = Series(event_yes_rsvp)
    #events["Waiting List"] = Series(waiting_list)
    events["Rsvp Limit"] = Series(rsvp_limit)
    events["Status"] = Series(status)
    return events

list = []
for group in ["DublinR","Cork-Ireland-R-Users-Group","pythonireland","PyLadiesDublin"]:
    for url in ["https://api.meetup.com/2/events.json/?group_urlname="+ group + "&key=95c3a7577a477a7d105a70197b5755","https://api.meetup.com/2/events.json/?group_urlname=" + group + "&status=past&key=95c3a7577a477a7d105a70197b5755"]:
        df = get_event_details(group, url)
        list.append(df)

#concatenate all DataFrames into one
all_events = pd.concat(list, ignore_index=True)
#we change that one or later on we'll have issues assogning group to plot our bubble graph
all_events.loc[all_events["Group Name"]=="Cork-Ireland-R-Users-Group","Group Name"]="Cork_Ireland_R_Users_Group"
for group in ["DublinR","Cork_Ireland_R_Users_Group","pythonireland","PyLadiesDublin"]:
    all_events.loc[(all_events["Rsvp Limit"]== 0) & (all_events["Group Name"] == group),"Rsvp Limit"]= math.ceil((all_events.groupby("Group Name").mean())["Number of yes_rsvp"][group])
all_events["Number Of Events"] =   Series(all_events.groupby(["Group Name",all_events["Date"].map(lambda d: dateutil.parser.parse(d).year)]).cumcount() + 1)
all_events["Date"] = Series([dateutil.parser.parse(all_events["Date"][i]).year for i in range(0,len(all_events))])


# Plotly Streaming Bubble chart
## Initiate Stream object

In [3]:
#get plotly credentials
stream_ids = tls.get_credentials_file()['stream_ids']

In [4]:
###make stream id object containing the stream id 
###link the stream id object to 'stream' key in the trace object of choice

#Set up a stream id graph object
#get the stream id from stream id list, only one needed here
stream_id = stream_ids[0]
#make a stream id object linking stream id to 'token' key
stream = Stream(token=stream_id)


## Initialize trace

In [5]:
# Initialize trace of streaming plot by embedding the unique stream_id
sizemode='area'
trace1 = Scatter(
    x=[],     # init. data
    y=[],
    text=[],  # init. hover text
    mode='markers',
    marker=Marker(  
        color=[],# init. marker color
        size=[],   # init. marker sizes
        opacity=0.6,          # partly transparent markers
        line=Line(width=0.0)  # remove marker borders
    ),  
    stream=stream  # (!) embed stream id, 1 per trace
)

data = Data([trace1])


## Layout and other functions

In [6]:
# (!) Set a reference for 'size' values (i.e. a Number_of_yes_rsvp-to-pixel scaling).


def hover_text(X, year):
     return 'Year: %d\<br> Group: %s\
     <br> Number of yes_rsvp: %d' % (year,X['Group Name'], X['Number of yes_rsvp'])
    
colors = dict(DublinR='#0000cc',
    Cork_Ireland_R_Users_Group='#1919ff',
    pythonireland='#00cc00',
    PyLadiesDublin='#00e500',
)

def make_anno(year):
    anno_text = '<b>Year: {} </b><br>Data source: Meetup API'.format(year)
    return dict(
        text=anno_text,  # set annotation text
        showarrow=False, # remove arrow 
        xref='paper',  # use paper coords
        yref='paper',  #  for both coordinates
        x=0.95,  # position's x-coord
        y=0.05,  #   and y-coord
        font=Font(size=14),    # increase font size (default is 12)
        bgcolor='#FFFFFF',     # white background
        borderpad=4            # space bt. border and text (in px)
)


# Set plot and axis titles
title = "Meetups in Ireland for Python and R"
x_title = "Rsvp Limit"
y_title = "Number of Events"

# Define a dictionary of axis style options
axis_style = dict(
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
        type = 'log',
        exponentformat = 'power',
        showexponent = 'all'
        #range = [13,200]
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
        range=[0,30]
    ),
    plot_bgcolor='#EFECEA',  # set plot color to grey
    hovermode = 'closest',
    showlegend = False,
    autosize = True,
)

# Make a figure object
fig = Figure(data=data, layout=layout)

# (@) Send fig to Plotly, initialize streaming plot, open new tab
unique_url = py.plot(fig, filename='kevo_bubbles')

## Open Stream

In [ ]:
#open the stream via the stream link object
s = py.Stream(stream_id)
s.open()

## Write to stream data

In [ ]:
#loop through all years in dataframe and plot every country one at a time 
#delay start of stream by 2 sec
time.sleep(1)

#extract years covered in our study. !!!SORTED!!!
years = Series(all_events["Date"][i] for i in range(0,len(all_events))).unique().tolist()
years = sorted(years)

while True: 
    for year in years: 
    #filter
        i_year = (all_events["Date"] == year)
        all_events_year = all_events[i_year]
        sizeref = all_events_year['Number of yes_rsvp'].max() / 1e2**1.75
    #sort data frame by number of events in ascending order
        all_events_year = all_events_year.sort_values(by="Number Of Events",ascending=True)
    #data dict to be streamed
        sdata = dict()
        slayout = dict(annotations=[make_anno(year)]) # layout dict. to be streamed
    
    #Iterate over the rows of a DataFrame as (index, Series) pairs.
        for i_X, X in all_events_year.iterrows():
            
            sdata['x'] = X['Rsvp Limit']
            sdata['y'] = X['Number Of Events']
            sdata['marker'] = dict(
                color=colors[X['Group Name']],  # color by group
                size=np.sqrt(X['Number of yes_rsvp']/sizeref)  # size by yes_rsvp
            )
            sdata['text'] = hover_text(X, year) # hover text info
        
            #write data and layout dict to stream every 0.1 sec 
            s.write(sdata,layout=slayout)
            time.sleep(0.01)

            #when all groups plotted, hold for 1 sec and flush 
        if year != years[-1]: #except for the last year
            time.sleep(0.3)
            sdata = dict(
                x=[],
                y=[],
                marker=dict(
                    color=[],
                    size=[]           
                ),
                text=[]
            )
            s.write(sdata,layout=slayout)
            #clear data
      

    #close stream
#s.close()   
